In [ ]:
import matplotlib.pyplot as plt
import torch
import os
import cv2
import numpy as np
import pandas as pd
import PIL 
import urllib
import torch.nn.functional as F
import torchvision as tv
import natsort
from torch.utils.data import SubsetRandomSampler
from tqdm import tqdm
from PIL import Image
from torch import nn
from torch import optim
from torchvision import datasets, transforms

#batch
batch_size = 64
os.chdir('/Users/hwangseoyeon/Desktop/3-1/AI')

In [ ]:
trans = transforms.Compose([
                                transforms.Resize(256),
                                transforms.RandomRotation(30), 
                                transforms.RandomVerticalFlip(0.5),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225]),
                               ])

In [ ]:
model = 'cpu'
if torch.cuda.is_available():
    model = 'cuda'

In [ ]:
#학습데이터 
train_data = tv.datasets.ImageFolder(root='/Users/hwangseoyeon/Desktop/3-1/AI/train', transform=trans)

In [ ]:
valid_size = 0.1  
num_train = len(train_data)
num_train = int(num_train)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler)
valid_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler)
len(train_dataloader),len(valid_dataloader)

In [ ]:
#학습
from efficientnet_pytorch import EfficientNet
model1 = EfficientNet.from_pretrained('efficientnet-b7', num_classes=5)

In [ ]:
import torch.optim as optim
model1.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(),lr=0.001)

In [ ]:
epochs = 180

valid_loss_min = np.Inf 
train_loss = torch.zeros(epochs)
valid_loss = torch.zeros(epochs)

train_acc = torch.zeros(epochs)
valid_acc = torch.zeros(epochs)


In [ ]:
for e in range(0, epochs):
    model1.train()
    for x, y in tqdm(train_dataloader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss[e] += loss.item()
        
        ps = F.softmax(logits, dim=1)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.reshape(top_class.shape)
        train_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
    
    train_loss[e] /= len(train_dataloader)
    train_acc[e] /= len(train_dataloader)
        
        
    with torch.no_grad(): 
        model1.eval()
        for x, y in tqdm(valid_dataloader):
            
            x, y = x.to(model), y.to(model)
            logits = model1(x)
            loss = criterion(logits, y)
            valid_loss[e] += loss.item()
            ps = F.softmax(logits, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.reshape(top_class.shape)
            valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
            
    valid_loss[e] /= len(valid_dataloader)
    valid_acc[e] /= len(valid_dataloader)
    
    if valid_loss[e] <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss[e]))
        torch.save(model.state_dict(), 'BestModel.pt')
        valid_loss_min = valid_loss[e]

In [ ]:
model1.load_state_dict(torch.load('/Users/hwangseoyeon/Desktop/3-1/AI/Model.pt'))

In [ ]:
test = natsort.natsorted(test_set)
test_trans = transforms.Compose([transforms.Resize(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485 , 0.456, 0.406], [0.229, 0.224 , 0.225])
                                    ])

In [ ]:
class test_data(torch.utils.data.Dataset):
    def __init__(self,imgpath,transform=test_transform):
        
        self.imgpath = imgpath
        self.transform = transform
        
    def __len__(self):
        return len(self.imgpath)
    
    def __getitem__(self,idx):
        x = Image.open(x).convert('RGB')

        if self.transform:
            x = self.transform(x)
        return x

In [ ]:
#테스트데이터 
test_data = test_data(test,trans=test_trans)
test_loader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=batch_size)

In [ ]:
model1.to(model)
model1.eval()
pred = []

#테스트 
with torch.no_grad():
  for img in tqdm(iter(test_loader)):
    img = img.to(model)
    pred = model1(img)
    f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
submission = pd.read_csv('/Users/hwangseoyeon/Desktop/3-1/AI/submission.csv')
submission['label'] = f_pred
submission.to_csv('/Users/hwangseoyeon/Desktop/3-1/AI/_submission.csv', index=False)